# Image Processing Pipeline **Setup**
--------------

## **OVERVIEW**

The first thing we need to be able to do is access the image files and interact with them (e.g., read the metadata).

#### &#x1F4D6; **How to:** 

Advance through each block of code by pressing `Shift`+`Enter`.

## **IMPORTS**

The convention with notebooks (and python in general) is to import the nescessary packages as the first thing.


We are using `napari` for visualization, and `scipy` `ndimage` and `skimage` for analyzing the image files.  The underlying data format are `numpy` `ndarrays` and tools from  Allen Institute for Cell Science.

> ###### 📝 **There are a few convences used here worth explanation.  Note the `imports.py` and `constants.py` files in the base level of the `infer_subc` module.  These provide shortcuts for keeping track of imports and constants.  cf. the bottom of the imports below.  A second thing to note is the use of the "magics" `%load_ext autoreload` `%autoreload 2`, which tells the notebook to reload any changes made in the source code of the module on change; hence, avoid re-executing the imports.**


#### &#x1F3C3; **Run code; no user input required**

&#x1F453; **FYI:** This code block loads all of the necessary python packages and functions you will need for this notebook. 

In [ ]:
# top level imports
from pathlib import Path
import os

import napari
import pandas as pd

from infer_subc.core.file_io import (read_czi_image,
                                    list_image_files,
                                    sample_input,
                                    export_ome_tiff)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## **LOAD AND READ IN IMAGE FOR PROCESSING**
Read the image and metadata into memory as an ndarray and dictionary from the `.czi` or `.tiff` files.

> #### ⬇️ **Downloading Sample Data**
>
> **If you would like to test this and the following notebooks in parts 1 and 2 with sample data, follow the steps below to download the raw sample images before beginning.**
>
> **Both of the example images were in the recent preprint titled [Neurons and astrocytes have distinct organelle signatures and responses to stress](https://www.biorxiv.org/content/10.1101/2024.10.30.621066v1)**
>
> Click the links to download the raw microscopy images
>
> ###### ⚠️ These files will begin downloading automatically when the links are clicked ⚠️
>
> Example Neuron File: [20230727_C2-121_unconditioned_well 10_cell 4_25nM TG_Linear unmixing_0_cmle.ome.tiff](https://www.ebi.ac.uk/biostudies/files/S-BIAD1445/Neuron%20and%20astrocyte%20organelle%20signatures%20dataset/Primary%20rat%20neuron%20data/Z-stacks/replicate3_C2-121/C2-121_deconvolution/20230727_C2-121_conditioned_well%204_cell%203_untreated_Linear%20unmixing_0_cmle.ome.tiff) (💾 2.16 GB)
>
> After downloading the example neuron file, move the .tiff image to the corresponding directory in the sample data folder.
>
>
> `infer-subc\sample_data\example_neuron\raw`
>
> Example Astrocyte File: [05052022_astro_control_2_Linear unmixing_0_cmle.ome](https://www.ebi.ac.uk/biostudies/files/S-BIAD1445/Neuron%20and%20astrocyte%20organelle%20signatures%20dataset/Primary%20rat%20astrocyte%20data/Z-stacks/replicate2_VD-0505/VD-0505_deconvolution/05052022_astro_control_2_Linear%20unmixing_0_cmle.ome.tiff) (💾 1.14 GB)
>
> After downloading the example astrocyte file, move the .tiff image to the corresponding directory in the sample data folder.
>
>
> `infer-subc\sample_data\example_astrocyte\raw`
>
>
> ###### 💾 You can now run all of the notebooks in infer-subc without editing the "&#x1F6D1; &#x270D; **User Input Required:**" fields. After running all of the notebooks in part 1, and completing quantification in part 2, the sample data folder will around 14 GB in size. Ensure that there is enough space on your drive to do so.

#### &#x1F6D1; &#x270D; **User Input Required:**

In [ ]:
### USER INPUT REQUIRED ###
# If using the sample data, select which cell type you would like analyze ("neuron" or "astrocyte"):
# If not using the sample data, set cell_type to None
sample_data_type = "neuron"

# If you are not using the sample data, please edit "USER SPECIFIED" as necessary.
## Define the path to the directory that contains the input image folder.
data_root_path = Path("USER SPECIFIED")

# Specify the file type of your raw data that will be analyzed. Ex) ".czi" or ".tiff"
im_type = "USER SPECIFIED"

## Specify which subfolder that contains the input data and the input data file extension
in_data_path = data_root_path / "USER SPECIFIED"

## Specify the output folder to save the segmentation outputs if.
## If its not already created, the code below will creat it for you
out_data_path = data_root_path / "USER SPECIFIED"

#### &#x1F3C3; **Run code; no user input required**

In [ ]:
# If sample_data_type is set to "neuron" or "astrocyte", then the sample data is used and the directories are set
if sample_data_type != None:
    raw_dirs = [Path(os.getcwd()).parents[1] / "sample_data" /  "example_neuron" / "raw",
                Path(os.getcwd()).parents[1] / "sample_data" /  "example_astrocyte" / "raw"]

# Copy the raw images to batch processing folder for quantification in part 2
    for dir in raw_dirs:
        f_list = list_image_files(dir,".tiff")
        for f in f_list:
            f_img, meta_f = read_czi_image(f)
            export_ome_tiff(data_in = f_img,
                        img_name = f.stem.split(".")[-2],
                        out_path = str(Path(os.getcwd()).parents[1]) + "/sample_data/batch_example/raw/",
                        meta_in = meta_f,
                        channel_names = meta_f['metadata']['aicsimage'].channel_names)

    data_root_path, im_type, in_data_path, out_data_path = sample_input(sample_data_type)

In [6]:
# Create the output directory to save the segmentation outputs in.
if not Path.exists(out_data_path):
    Path.mkdir(out_data_path)
    print(f"making {out_data_path}")

# Create a list of the file paths for each image in the input folder. Select test image path.
img_file_list = list_image_files(in_data_path,im_type)
pd.set_option('display.max_colwidth', None)
pd.DataFrame({"Image Name":img_file_list})

,Image Name
0,c:\Users\redre\Documents\CohenLab\MSI-3D-analysis\infer-subc\sample_data\example_astrocyte\raw\05052022_astro_control_2_Linear unmixing_0_cmle.ome.tiff


#### &#x1F6D1; &#x270D; **User Input Required:**

In [7]:
# Based on the output table above, select the file you'd like to examine.
test_img_n = 0

#### &#x1F3C3; **Run code; no user input required**

In [8]:
# Read in the image and metadata as an ndarray and dictionary from the test image selected above. 
test_img_name = img_file_list[test_img_n]
img_data,meta_dict = read_czi_image(test_img_name)

# Define some of the metadata features.
channel_names = meta_dict['name']
img = str(test_img_name.stem)
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']

print("Metadata information")
print(f"Channel names: {channel_names}")
print(f"Image name: {img}")
print(f"Scale: {scale}")
print(f"Channel axis: {channel_axis}")

# Visualize the image in the Napari window
viewer = napari.Viewer(title = f"Example Image: {img}")
viewer.add_image(img_data)
print("\nProceed to Napari window to view your selected image.")

Metadata information
Channel names: ['20230727_C2-121_conditioned_well 4_cell 3_untreated_Linear unmixing_0_cmle.ome :: Channel:0', '20230727_C2-121_conditioned_well 4_cell 3_untreated_Linear unmixing_0_cmle.ome :: Channel:1', '20230727_C2-121_conditioned_well 4_cell 3_untreated_Linear unmixing_0_cmle.ome :: Channel:2', '20230727_C2-121_conditioned_well 4_cell 3_untreated_Linear unmixing_0_cmle.ome :: Channel:3', '20230727_C2-121_conditioned_well 4_cell 3_untreated_Linear unmixing_0_cmle.ome :: Channel:4', '20230727_C2-121_conditioned_well 4_cell 3_untreated_Linear unmixing_0_cmle.ome :: Channel:5']
Image name: 20230727_C2-121_conditioned_well 4_cell 3_untreated_Linear unmixing_0_cmle.ome
Scale: (0.410594, 0.079947, 0.079947)
Channel axis: 0

Proceed to Napari window to view your selected image.


--------------
### NEXT:  INFER MASKS

proceed to:

[**Alternative Workflow (A)**](./1.1a_infer_masks_from-composite_single_cell.ipynb) - an alternative workflow for images with only cytoplasmic organelles, NO **nuclei** or **cell membrane** makers, one cell per field of view

[**Alternative Workflow (B)**](./1.1b_infer_masks_from-composite_multiple-cells.ipynb) - an alternative workflow for images with only cytoplasmic organelles, NO **nuclei** or **cell membrane** makers, and more than one cell per field of view

> ###### **Sample Data not yet applicable with 1**

[**Default Workflow (1)**](./1.1_infer_masks_from-composite_with_nuc.ipynb) - for images with cytoplasmic organelles, a **nuclei** marker, no **cell membrane** makers, and more than one cell per field of view
